# ÍNDICES e PARTICIONAMENTO

Na aula anterior estudamos os conceitos de **dependências funcionais** e **normalização**. O conhecimento das formas normais propicia verificar a possibilidade de realizar alterações no design das bases de dados, com o objetivo de **evitar repetições** e **recuperar informações de forma fácil**.

Entretanto, mesmo obedecendo (ou não) às formas normais, em certas situações a performance de nossas queries em produção se torna sofrível. Nesta aula, veremos alguns recursos que podemos utilizar para melhorar o desempenho das consultas produzidas.

Este é um tema bastante relevante. Um engenheiro sem o devido conhecimento pode recomendar, por exemplo, a compra de mais servidores (físicos ou na AWS), quando a aplicação dos conceitos vistos nesta aula poderiam gerar ganhos de múltiplas vezes no tempo de execução de queries!

## Instalação da base

Vamos utilizar a base de dados sintética disponível no Blackboard. Execute o script `script_elet_001.sql` para criar a base de dados. Este script apenas faz a **DDL**, os dados gerão gerados de forma aleatória neste notebook.

## Import das bibliotecas

Vamos realizar o import das bibliotecas.

In [1]:
import mysql.connector
import os
import random
from functools import partial
from datetime import datetime, timedelta
from dotenv import load_dotenv
from IPython.display import clear_output

E vamos criar nosso HELPER de conexão com o banco! Perceba que, uma vez configurado o `.env` não precisaremos mais informar usuários, senhas e URLs!

In [2]:
load_dotenv(override=True)

def get_connection_helper():

    def run_db_query(connection, query, args=None, verbose=True):
        with connection.cursor() as cursor:
            if verbose:
                print("Executando query:")
            cursor.execute(query, args)
            for result in cursor:
                if verbose:
                    print(result)

    connection = mysql.connector.connect(
        host=os.getenv("MD_DB_SERVER"),
        user=os.getenv("MD_DB_USERNAME"),
        password=os.getenv("MD_DB_PASSWORD"),
        database="eletrobeer",
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

## Gerar dados para a base

Vamos gerar alguns valores aleatórios e inserir na base de dados `eletrobeer`.

Primeiro, defina a quantidade de linhas a serem inseridas:

In [3]:
QTDE_PED = 4000000

Vamos definir algumas funções auxiliares

In [4]:
def get_random_date():
    start_date = datetime(2015, 1, 1)
    end_date = datetime(2022, 12, 31)

    delta = end_date - start_date
    random_date = start_date + timedelta(days=random.randint(0, delta.days))

    return  random_date.strftime("%Y-%m-%d")

def get_cidade_uf():
    cidades = [
        ["São Paulo", "SP"],
        ["Campinas", "SP"],
        ["Ribeirão Preto", "SP"],
        ["São Roque", "SP"],
        ["Rio de Janeiro", "RJ"],
        ["Macaé", "RJ"],
        ["Angra dos Reis", "RJ"],
    ]

    cidade, uf = random.choice(cidades)
    return cidade, uf

def insert_pedidos(connection, pedidos):
    cursor = connection.cursor()
    sql = """INSERT INTO eletrobeer.pedido
    (id_pedido, id_cliente, data_criacao, qtde_itens, valor_total, cidade_entrega, uf_entrega)
    VALUES (%s, %s, %s, %s, %s, %s, %s)"""
    cursor.executemany(sql, pedidos)
    cursor.close()

Garantir que a tabela `pedido` está vazia

In [5]:
db("DELETE FROM eletrobeer.pedido")

Executando query:


Então, geramos os dados aleatórios.

**<span style="color:red">Atenção</span>**: Este processo pode demorar alguns minutos! A cada aproximadamente 5 segundos, `BATCH_SIZE` inserções devem ser realizadas, com o retorno da mensagem `Completou 200000`, `Completou 400000` e assim por diante!

**Dica:** enquanto executa a próxima célula, apenas por curiosidade, leia o seguinte material https://dev.mysql.com/doc/refman/8.0/en/optimizing-innodb-bulk-data-loading.html.

In [6]:
BATCH_SIZE = 200000

pedidos = []

for id_pedido in range(1, QTDE_PED + 1):
    id_cliente = random.randint(1000, 10000)
    qtde_itens = random.randint(1, 200)
    data_criacao = get_random_date()
    valor_total = random.uniform(15.0, 300000)
    cidade, uf = get_cidade_uf()

    pedido = (id_pedido, id_cliente, data_criacao, qtde_itens, valor_total, cidade, uf)
    pedidos.append(pedido)

    if len(pedidos) == BATCH_SIZE:
        insert_pedidos(connection, pedidos)
        pedidos = []

    if id_pedido % BATCH_SIZE == 0:
        clear_output()
        print(f"Completou {id_pedido}")

if pedidos:
    insert_pedidos(connection, pedidos)

connection.commit()

Completou 4000000


E vamos fazer `commit` para garantir que os dados foram salvos!

In [7]:
connection.commit()

## Consultando a base

Vamos fazer algumas consultas. Como nosso objetivo é avaliar a performance das queries, precisamos analisar o tempo que cada query necessita para executar.

Uma opção é fazer o cálculo direto no Python:

In [8]:
import time

start_time = time.time() # get current time

db("SELECT COUNT(*) FROM pedido")

end_time = time.time() # get current time again

time_spent = end_time - start_time # calculate time spent

print("Time spent:", time_spent, "seconds")

Executando query:


(4000000,)
Time spent: 20.13348150253296 seconds


Entretanto, desta forma estamos considerando o tempo total, incluindo o tempo gasto pelas funções do próprio Python. Talvez não seja um tempo significativo, mas como poderia ser, melhor evitar e considerar o tempo isolado: apenas o que foi gasto de fato para a query executar (após ter sido recebida pelo RDBMS MySQL).

Para isto, vamos ativar **profiling**:

In [9]:
db("SET profiling = 1;", verbose=False)

**Obs**: quando quiser desativar, utilize

```mysql
SET profiling = 0;
```

Então podemos executar algum **SQL**

In [10]:
db("SELECT COUNT(*) FROM pedido")

Executando query:


(4000000,)


E ver, no segundo elemento de cada tupla, qual o tempo gasto para a query ser executada.

In [11]:
db("SHOW PROFILES;")

Executando query:
(1, 17.2627995, 'SELECT COUNT(*) FROM pedido')


Então, podemos executar novas queries e ver o seu tempo consumido:

**Dica**: aqui, `verbose=False` executará a query, mas não exibirá o resultado na saída. Podemos fazer assim quando nosso interesse é em apenas ter o tempo da query e não o resultado

In [12]:
sql = "SELECT COUNT(*) as QTDE_LINHAS FROM pedido"

db(sql, verbose=False)
db("SHOW PROFILES;")

Executando query:
(1, 17.2627995, 'SELECT COUNT(*) FROM pedido')
(2, 16.90179775, 'SELECT COUNT(*) as QTDE_LINHAS FROM pedido')


Utilize `SET PROFILING_HISTORY_SIZE = ???;`, trocando `???` pelo **número de queries** que quer ver no resultado dos profiles, para limitar a exibição as últimas **número de queries**.

In [13]:
db("SET PROFILING_HISTORY_SIZE = 2;", verbose=False)
db("SHOW PROFILES;")

Executando query:
(2, 16.90179775, 'SELECT COUNT(*) as QTDE_LINHAS FROM pedido')
(3, 0.000792, 'SET PROFILING_HISTORY_SIZE = 2')


Vamos deixar configurado para `5`. Você pode alterar quando quiser!

In [14]:
db("SET PROFILING_HISTORY_SIZE = 5;", verbose=False)
db("SHOW PROFILES;")

Executando query:
(2, 16.90179775, 'SELECT COUNT(*) as QTDE_LINHAS FROM pedido')
(3, 0.000792, 'SET PROFILING_HISTORY_SIZE = 2')
(4, 0.00013275, 'SET PROFILING_HISTORY_SIZE = 5')


### Explorando novos exemplos

Vamos executar algumas consultas e verificar seus tempos de execução.

**Dica**: leia cada query e tente entender o que ela faz!

In [15]:
sql1 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega = "São Paulo";"""

sql2 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega = "Rio de Janeiro";"""

sql3 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega IN ("Rio de Janeiro", "São Paulo");"""

sql4 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega LIKE "São%";"""

sql5 = r"""SELECT count(*) FROM pedido p
WHERE p.cidade_entrega LIKE "%de%";"""

db(sql1)
db(sql2)
db(sql3)
db(sql4)
db(sql5)

db("SHOW PROFILES;")

Executando query:


(571337,)
Executando query:
(571438,)
Executando query:
(1142775,)
Executando query:
(1141868,)
Executando query:
(571438,)
Executando query:
(5, 2.08327525, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega = "São Paulo"')
(6, 2.72379525, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega = "Rio de Janeiro"')
(7, 5.98593275, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega IN ("Rio de Janeiro", "São Paulo")')
(8, 4.9428245, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega LIKE "São%"')
(9, 15.02367675, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega LIKE "%de%"')


### MEGADADOS?!

Podemos perceber que esta base tem (deveria ter!!!) 4 milhões de linhas.

In [16]:
sql = "SELECT COUNT(*) as QTDE_LINHAS FROM pedido"

db(sql)

Executando query:
(4000000,)


No resultados do `SHOW PROFILES`, vemos que o tempo é quase zero. Mas não se engane, estamos mantendo um ambiente controlado, com apenas uma tabela e realizando queries simples o suficiente para entendermos o que está acontecendo.

Em uma situação do mercado profissional, prepare-se para trabalhar com milhões ou bilhões de linhas em dezenas ou centenas de tabelas, construindo queries de centenas e até milhares de linhas que rodam milhares ou milhões de vezes! As situações de dependência entre múltiplas colunas e tabelas gerará muitas situações onde os conceitos da aula poderão ser aplicados.

**Dica**: No MySQL Workbench, dê botão direito na tabela / Table Inspector. Abra o explorer e confira o tamanho do arquivo contido no **Data path**! Novamente, em uma situação de mercado, espere gigabytes!

### Índices

Índices são estruturas de dados que facilitam a localização de informação no banco de dados.

**Obs**: Link para simular `BTREE` https://www.cs.usfca.edu/~galles/visualization/BTree.html

Para criar um índice, vamos utilizar a sintaxe:
```mysql
CREATE INDEX index_name [index_type] 
 ON tbl_name (index_col_name,...)
```

Por exemplo:

```mysql
-- Por padrão, o index será BTREE
CREATE INDEX pedido_cidade_entrega_IDX
 ON eletrobeer.pedido (cidade_entrega);
```

É importante lembrar que nem todos os engines suportam índice **HASH**. A engine padrão da versão que utilizamos (**InnoDB**) não suporta!

In [17]:
db("SHOW ENGINES;")

Executando query:
('MEMORY', 'YES', 'Hash based, stored in memory, useful for temporary tables', 'NO', 'NO', 'NO')
('MRG_MYISAM', 'YES', 'Collection of identical MyISAM tables', 'NO', 'NO', 'NO')
('CSV', 'YES', 'CSV storage engine', 'NO', 'NO', 'NO')
('FEDERATED', 'NO', 'Federated MySQL storage engine', None, None, None)
('PERFORMANCE_SCHEMA', 'YES', 'Performance Schema', 'NO', 'NO', 'NO')
('MyISAM', 'YES', 'MyISAM storage engine', 'NO', 'NO', 'NO')
('InnoDB', 'DEFAULT', 'Supports transactions, row-level locking, and foreign keys', 'YES', 'YES', 'YES')
('ndbinfo', 'NO', 'MySQL Cluster system information storage engine', None, None, None)
('BLACKHOLE', 'YES', '/dev/null storage engine (anything you write to it disappears)', 'NO', 'NO', 'NO')
('ARCHIVE', 'YES', 'Archive storage engine', 'NO', 'NO', 'NO')
('ndbcluster', 'NO', 'Clustered, fault-tolerant tables', None, None, None)


Vamos criar um índice na coluna `cidade_entrega` e repetir as queries.

Mas antes, vamos consultar se existe algum índice atualmente nesta tabela!

In [18]:
db("SHOW INDEX FROM pedido;")

Executando query:
('pedido', 0, 'PRIMARY', 1, 'id_pedido', 'A', 1872264, None, None, '', 'BTREE', '', '', 'YES', None)
('pedido', 0, 'PRIMARY', 2, 'data_criacao', 'A', 1873948, None, None, '', 'BTREE', '', '', 'YES', None)
('pedido', 1, 'pedido_cidade_entrega_IDX', 1, 'cidade_entrega', 'A', 7, None, None, 'YES', 'BTREE', '', '', 'YES', None)


Perceba que já existe um índice. Por que ele está aí se ainda não criamos nenhum?!

<div class="alert alert-success">

Sua resposta AQUI!

</div>

<a href="#" title="O índice foi criado para a coluna que é chave primária! Isto é feito por padrão, uma vez que a chave acaba sendo utilizada em diversas consultas e joins.">Pare o  mouse aqui para ver a resposta</a>

Então criamos o índice

In [19]:
db("CREATE INDEX pedido_cidade_entrega_IDX ON eletrobeer.pedido (cidade_entrega);")

Executando query:


ProgrammingError: 1061 (42000): Duplicate key name 'pedido_cidade_entrega_IDX'

E conferimos novamente os índices existentes

In [ ]:
db("SHOW INDEX FROM pedido;")

Executando query:
('pedido', 0, 'PRIMARY', 1, 'id_pedido', 'A', 3822104, None, None, '', 'BTREE', '', '', 'YES', None)
('pedido', 1, 'pedido_cidade_entrega_IDX', 1, 'cidade_entrega', 'A', 6, None, None, 'YES', 'BTREE', '', '', 'YES', None)


Caso queira ver o título das coluas, execute o `SHOW INDEX FROM pedido;` direto no MySQL WorkBench!

Agora repetimos as queries. Compare o tempo necessários para suas execuções.

In [ ]:
sql1 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega = "São Paulo";"""

sql2 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega = "Rio de Janeiro";"""

sql3 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega IN ("Rio de Janeiro", "São Paulo");"""

sql4 = """SELECT count(*) FROM pedido p
WHERE p.cidade_entrega LIKE "São%";"""

sql5 = r"""SELECT count(*) FROM pedido p
WHERE p.cidade_entrega LIKE "%de%";"""

db(sql1)
db(sql2)
db(sql3)
db(sql4)
db(sql5)

db("SHOW PROFILES;")

Executando query:


(569970,)
Executando query:
(571307,)
Executando query:
(1141277,)
Executando query:
(1142128,)
Executando query:
(571307,)
Executando query:
(15, 0.524418, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega = "São Paulo"')
(16, 0.60939575, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega = "Rio de Janeiro"')
(17, 1.322551, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega IN ("Rio de Janeiro", "São Paulo")')
(18, 0.95133075, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega LIKE "São%"')
(19, 1.59838125, 'SELECT count(*) FROM pedido p\nWHERE p.cidade_entrega LIKE "%de%"')


Compare o tempo **antes** *versus* **depois** da criação do índice. Apesar de não estamos analisando uma amostra de tamanho 1 (CDados manda oi!), é esperado que obtenha uma melhora de múltiplas vezes em algumas queries, mas em outras nem tanto. Você consegue explicar o por que?!

**Dica**: https://dev.mysql.com/doc/refman/8.0/en/index-btree-hash.html#btree-index-characteristics

<div class="alert alert-success">

Sua resposta AQUI!

</div>

#### Testando com outros campos e queries!

Vamos testar com com outros campos e queries!

In [ ]:
db("SET PROFILING_HISTORY_SIZE = 4;", verbose=False)

In [ ]:
sql1 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens = 8;"""

sql2 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens < 5;"""

sql3 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens BETWEEN 5 AND 20;"""

sql4 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens IN (8, 15, 20, 45);"""

db(sql1)
db(sql2)
db(sql3)
db(sql4)

db("SHOW PROFILES;")

Executando query:


(19853,)
Executando query:
(79579,)
Executando query:
(319692,)
Executando query:
(79675,)
Executando query:
(21, 1.701666, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens = 8')
(22, 2.18712125, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens < 5')
(23, 2.5228705, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens BETWEEN 5 AND 20')
(24, 2.3360255, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens IN (8, 15, 20, 45)')


Conferindo os índices atuais

In [ ]:
db("SHOW INDEX FROM pedido;")

Executando query:
('pedido', 0, 'PRIMARY', 1, 'id_pedido', 'A', 3822104, None, None, '', 'BTREE', '', '', 'YES', None)
('pedido', 1, 'pedido_cidade_entrega_IDX', 1, 'cidade_entrega', 'A', 6, None, None, 'YES', 'BTREE', '', '', 'YES', None)


Crie um index **BTREE** baseado na coluna `qtde_itens`

In [ ]:
db("-- SUA QUERY PARA CRIAR O INDEX AQUI!!!")

Executando query:


Conferindo os índices

In [ ]:
db("SHOW INDEX FROM pedido;")

Executando query:
('pedido', 0, 'PRIMARY', 1, 'id_pedido', 'A', 3822104, None, None, '', 'BTREE', '', '', 'YES', None)
('pedido', 1, 'pedido_cidade_entrega_IDX', 1, 'cidade_entrega', 'A', 6, None, None, 'YES', 'BTREE', '', '', 'YES', None)


Repetindo as consultas

In [ ]:
sql1 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens = 8;"""

sql2 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens < 5;"""

sql3 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens BETWEEN 5 AND 20;"""

sql4 = """SELECT count(*) FROM pedido p
WHERE p.qtde_itens IN (8, 15, 20, 45);"""

db(sql1)
db(sql2)
db(sql3)
db(sql4)

db("SHOW PROFILES;")

Executando query:


(19853,)
Executando query:
(79579,)
Executando query:
(319692,)
Executando query:
(79675,)
Executando query:
(28, 2.445799, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens = 8')
(29, 2.15931025, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens < 5')
(30, 2.483778, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens BETWEEN 5 AND 20')
(31, 2.341054, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens IN (8, 15, 20, 45)')


Compare o tempo **antes** *versus* **depois** da criação do índice. Emocionante, não?!

<div class="alert alert-success">

Sua resposta AQUI!

</div>

## Particionamento

Particionar é dividir as tabelas de um banco de dados em partes menores.

Permite distribuir o banco de dados em vários nós ou HDs diferentes, aumentando o desempenho em situações de acesso concorrente intenso (que não é o nosso caso).

Leia mais em https://dev.mysql.com/doc/refman/8.0/en/partitioning-pruning.html

Veja um exemplo de particionamento:

In [ ]:
sql1 = """
SELECT 
    YEAR(p.data_criacao), AVG(valor_total) AS media
FROM
    pedido p
WHERE YEAR(p.data_criacao) > 2020
GROUP BY YEAR(p.data_criacao)
ORDER BY YEAR(p.data_criacao) ASC;"""

db(sql1)

db("SHOW PROFILES;")

Executando query:


(2021, Decimal('150124.908691'))
(2022, Decimal('149772.414290'))
Executando query:
(29, 2.15931025, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens < 5')
(30, 2.483778, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens BETWEEN 5 AND 20')
(31, 2.341054, 'SELECT count(*) FROM pedido p\nWHERE p.qtde_itens IN (8, 15, 20, 45)')
(32, 3.20672375, 'SELECT \n    YEAR(p.data_criacao), AVG(valor_total) AS media\nFROM\n    pedido p\nWHERE YEAR(p.data_criacao) > 2020\nGROUP BY YEAR(p.data_criacao)\nORDER BY YEAR(p.data_criacao) ASC')


Para separar por ano, precisaremos fazer com que a `data_criacao` seja parte da chave primária.

In [ ]:
sql = """
ALTER TABLE pedido
DROP PRIMARY KEY;"""

db(sql)

Executando query:


In [ ]:
sql = """
ALTER TABLE pedido
ADD PRIMARY KEY (id_pedido, data_criacao);"""

db(sql)

Executando query:


Então, separamos a tabela `pedido` em quatro partições

In [ ]:
sql = """
ALTER TABLE pedido
PARTITION BY RANGE(YEAR(data_criacao))
(
    PARTITION p0 VALUES LESS THAN (2016),
    PARTITION p1 VALUES LESS THAN (2018),
    PARTITION p2 VALUES LESS THAN (2020),
    PARTITION p3 VALUES LESS THAN MAXVALUE
);"""

db(sql)

Executando query:


Executando a query novamente

In [ ]:
sql1 = """
SELECT 
    YEAR(p.data_criacao), AVG(valor_total) AS media
FROM
    pedido p
WHERE YEAR(p.data_criacao) > 2020
GROUP BY YEAR(p.data_criacao)
ORDER BY YEAR(p.data_criacao) ASC;"""

db(sql1)

db("SHOW PROFILES;")

Executando query:
(2021, Decimal('150124.908691'))
(2022, Decimal('149772.414290'))
Executando query:
(33, 57.51880525, 'ALTER TABLE pedido\nDROP PRIMARY KEY')
(34, 45.67802325, 'ALTER TABLE pedido\nADD PRIMARY KEY (id_pedido, data_criacao)')
(35, 65.2390475, 'ALTER TABLE pedido\nPARTITION BY RANGE(YEAR(data_criacao))\n(\n    PARTITION p0 VALUES LESS THAN (2016),\n    PARTITION p1 VALUES LESS THAN (2018),\n    PARTITION p2 VALUES LESS THAN (2020),\n    PARTITION p3 VALUES LESS THAN MAXVALUE\n)')
(36, 4.844234, 'SELECT \n    YEAR(p.data_criacao), AVG(valor_total) AS media\nFROM\n    pedido p\nWHERE YEAR(p.data_criacao) > 2020\nGROUP BY YEAR(p.data_criacao)\nORDER BY YEAR(p.data_criacao) ASC')


Anote abaixo suas considerações sobre particionar tabelas!

<div class="alert alert-info">

Suas observações AQUI!

</div>

## Exercícios

**Exercício 1**: Explique por que uma hash table é:

**a)** Boa para buscas por valor exato?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**b)** Ruim para buscas por faixas de valor?

**Dicas**:
- Tente pensar por alguns minutos como as hash tables funcionam
- Se travar, peça ajuda aos professores ou pergunte ao ChatGPT.

<div class="alert alert-success">

Sua resposta AQUI!

</div>

<a href="#gab_ex1">Click para ver a resposta</a>

**Exercício 2**: Por que os índices de bancos de dados relacionais utilizam `B-tree` e suas variantes ao invés de uma árvore binária balanceada?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**Exercício 3**: O professor demonstrou a construção de uma `B-tree`. Pesquise o que são as `B+-trees` e responda:

**Obs:**
- O `+` representa um **plus**. Já o `-` é só um traço separador!
- Você pode simular a versão **plus** aqui https://www.cs.usfca.edu/~galles/visualization/BPlusTree.html

**a)** Qual a diferença entre a `B-tree` e a versão plus?!

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**b)** O MySQL utiliza `B-tree` ou `B+-trees`?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**Exercício 4**: Explique por que uma B-tree é:

**a)** Razoável para buscas por valor exato?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**b)** Boa para buscas por faixa de valor?

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**Exercício 5**: Pense em situações onde o **particionamento vertical** é benéfico, e onde é problemático.

<div class="alert alert-success">

Sua resposta AQUI!

</div>

**Exercício 6**: Pense em situações onde o **particionamento horizontal** é benéfico, e onde é problemático.

<div class="alert alert-success">

Sua resposta AQUI!

</div>

## Conexão

Vamos fechar a conexão e finalizamos por hoje!

In [ ]:
connection.close()

## Referências
- OLIVEIRA, C. H. P, SQL: Curso Prático, Novatec, 2002 CAP 4
- SILBERSCHATZ, A.; KORTH, H. F.; SUDARSHAN, S. DATABASE SYSTEM CONCEPTS, SEVENTH EDITION CAP 4.6
- https://jasimabasheer.com/posts/btrees

## Gabarito

**<div id="gab_ex1">Exercício 1</div>**
<div class="alert alert-warning">

**a)** Complexide de busca O(1) 😍

**b)** Não mantém relação de ordem 😭
    
</div>